In [1]:
# Default libraries: pandas, numpy
# import beakerx
import cProfile
import collections
import datetime

import feather
import itertools
import math
import os
import pickle
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
# import urbangrammar_graphics as ugg

from IPython.display import Markdown as md
from collections import Counter
from clustergram import Clustergram
from datetime import datetime
from dotenv import load_dotenv
from itertools import groupby

from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier, BaggingClassifier
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, classification_report
from sklearn.model_selection import train_test_split, cross_validate, learning_curve, GridSearchCV, RandomizedSearchCV, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.tree import DecisionTreeClassifier

from xgboost import XGBClassifier

pd.options.display.float_format = "{:.2f}".format
# plt.rcParams.update({'font.size': 18})

https://developer.spotify.com/documentation/web-api/
https://developer.spotify.com/
https://developer.spotify.com/dashboard/login

**Objectives:**
- recommend the next song for an user base on his current playlist

**Assumptions:**
- tracks of a given playlist are the favorite songs of the user

**Methodology:**
1. Collaborative filtering
2. Initial sample for each user will take 70% of the tracks in the playlist (check if the sorting of the musics in the playlist has any information - date of inclusion, user sorting, etc) and then use the remaining 30% of the tracks as test.
3. **If we have time:** Hybrid model by adding *Content based filtering* for playlists with few tracks
    - Track features: genre, time, top_track, etc.
**Concerns:**
- cold start: playlists with few tracks
- rare tracks: treatment to be applied to the tracks with low frequency on the database
- only positive rating: we will have only tracks that the user liked, we do not have any data about tracks that they did not like.


**Python Libraries:**
- Sagemaker
- Elasticsearch
- PySpark

In [2]:
load_dotenv()
SPOTIPY_API = os.getenv('SPOTIPY_API')
SPOTIPY_API_KEY = os.getenv('SPOTIPY_API_KEY')
scope=''

# https://spotipy.readthedocs.io/en/latest/#
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_API, client_secret=SPOTIPY_API_KEY)

# help(sp)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
def get_playlist_detail(user_id, playlist_id):
    l_artists = []
    l_albums = []
    l_tracks = []
    l_popularity = []
    l_artists_id = []
    l_albums_id = []
    l_tracks_id = []
    
    playlist = sp.user_playlist(user_id, playlist_id)
    
    for item in playlist['tracks']['items']:
        group = []
        group_id = []
        for artist in item['track']['album']['artists']:
            group.append(artist['name'])
            group_id.append(artist['id'])
        l_artists.append(group)
        l_artists_id.append(group_id)
        l_albums.append(item['track']['album']['name'])
        l_tracks.append(item['track']['name'])
        l_popularity.append(item['track']['popularity'])
        
        l_albums_id.append(item['track']['album']['id'])
        l_tracks_id.append(item['track']['id'])
        
    return user_id, playlist_id, l_artists, l_albums, l_tracks, l_popularity, l_artists_id, l_albums_id, l_tracks_id;

# user_id, playlist_id, l_artists, l_albums, l_tracks, l_popularity, l_artists_id, l_albums_id, l_tracks_id = get_playlist_detail('spotify', '37i9dQZF1DWXjlWxUbm84A')
# user_id, playlist_id, l_artists, l_albums, l_tracks, l_popularity, l_artists_id, l_albums_id, l_tracks_id

In [4]:
t_start = time.time()
data_path = os.path.dirname(os.getcwd()) + '\\data\\external\\raw_data.pkl'
if os.path.exists(data_path):
    df = pd.read_pickle(data_path)
else:
#     print('ops')
    n = 50
    p_limit = 50 # limit of playlists max = 50

    user_id = []
    playlist_id = []
    artists = []
    albums = []
    tracks = []
    popularity = []
    artists_id = []
    albums_id = []
    tracks_id = []

    for y in range(2018,2019,1):
        txt = 'year:' + str(y)
        print(txt)
        for i in range(0,n,1):
#             if i-1 % 100 == 0: print(i)
            result = sp.search(q=txt, type='playlist', limit=p_limit, offset=i) #,market='US'
#             print(str(y) + ": " + str(i))
            for user in result['playlists']['items']:
                pid = user['id']
                uid = user['owner']['id']
#                 user_id.append(uid)
#                 playlist_id.append(pid)
                p_user_id, p_playlist_id, p_artists, p_albums, p_tracks, p_popularity, p_artists_id, p_albums_id, p_tracks_id = get_playlist_detail(uid,pid)

                user_id.append(p_user_id)
                playlist_id.append(p_playlist_id)
                artists.append(p_artists)
                albums.append(p_albums)
                tracks.append(p_tracks)
                popularity.append(p_popularity)
                artists_id.append(p_artists_id)
                albums_id.append(p_albums_id)
                tracks_id.append(p_tracks_id)
#             if i-1 % 100 == 0: print('tot:' + str(len(playlist_id)))

    # intialise data of lists.
    data = {
        'user_id': user_id,
        'playlist_id': playlist_id,
        'tracks_id': tracks_id,
        'tracks': tracks,
        'popularity': popularity,
        'artists_id': artists_id,
        'artists': artists,
        'albums_id': albums_id,
        'albums': albums
    }

    # Create DataFrame
    df = pd.DataFrame(data)
    df.head()

    # Save the data
    df.to_pickle(data_path)

t_end = time.time()
np.abs(t_start - t_end)

0.5620009899139404

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2450 entries, 0 to 2449
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      2450 non-null   object
 1   playlist_id  2450 non-null   object
 2   tracks_id    2450 non-null   object
 3   tracks       2450 non-null   object
 4   popularity   2450 non-null   object
 5   artists_id   2450 non-null   object
 6   artists      2450 non-null   object
 7   albums_id    2450 non-null   object
 8   albums       2450 non-null   object
dtypes: object(9)
memory usage: 172.4+ KB


In [6]:
def single_list(my_list):
    single = []
    for e in my_list:
        single.extend(e)
    return single

def single_list_nested(my_list):
    single = []
    for e in my_list:
        for a in e:
            single.extend(a)
    return single

def sorted_list(xs):
    counts = collections.defaultdict(int)
    for x in xs:
        counts[x] += 1
    return sorted(counts.items(), reverse=True, key=lambda tup: tup[1])

In [7]:
s_tracks_id = single_list(df['tracks_id'])
s_tracks = single_list(df['tracks'])
s_popularity = single_list(df['popularity'])

s_artists = single_list_nested(df['artists'])
s_artists_id = single_list_nested(df['artists_id'])

s_albums = single_list(df['albums'])
s_albums_id = single_list(df['albums_id'])

data = {
    'tracks_id': s_tracks_id,
    'tracks': s_tracks,
    'popularity': s_popularity
}
df_track = pd.DataFrame(data)
# print(df_track.head())

data = {
    'artists_id': s_artists_id,
    'artists': s_artists
}
df_artist = pd.DataFrame(data)
# print(df_artist.head())


data = {
    'albums_id': s_albums_id,
    'albums': s_albums
}
df_album = pd.DataFrame(data)
# print(df_album.head())

In [8]:
tracks_rank = df_track.groupby(['tracks_id','tracks']).agg(
    popularity=('popularity','max'),
    tracks_count=('tracks_id','count')
    )

min_max = MinMaxScaler()
tracks_rank['count_scaled'] = min_max.fit_transform(tracks_rank[["tracks_count"]])

tracks_rank["rank"] = tracks_rank['tracks_count'].rank(method='average',ascending=False)

tracks_rank = tracks_rank.sort_values('tracks_count',ascending=False)

tracks_rank

,,popularity,tracks_count,count_scaled,rank
tracks_id,tracks,,,,
2xLMifQCjDGFmkHkpNLD9h,SICKO MODE,83,236,1.00,1.00
2G7V7zsVDxg1yRsu7Ew9RJ,In My Feelings,76,213,0.90,2.00
45Egmo7icyopuzJN0oMEdk,Love Lies (with Normani),74,203,0.86,3.00
3ee8Jmje8o58CHK66QrVC2,SAD!,84,199,0.84,4.00
10Igtw8bSDyyFs7KIsKngZ,Freaky Friday (feat. Chris Brown),71,196,0.83,5.00
...,...,...,...,...,...
6K8qKeWo5MsFED7wCR6Kop,Sing,72,1,0.00,6479.50
6KBYk8OFtod7brGuZ3Y67q,Misery,60,1,0.00,6479.50
2iuZJX9X9P0GKaE93xcPjk,Sugar,80,1,0.00,6479.50


In [9]:
album_rank = df_album.groupby(['albums_id','albums']).agg(album_count=('albums_id','count'))

min_max = MinMaxScaler()
album_rank['count_scaled'] = min_max.fit_transform(album_rank[["album_count"]])

album_rank["rank"] = album_rank['album_count'].rank(method='average',ascending=False)

album_rank = album_rank.sort_values('album_count',ascending=False)

album_rank

,,album_count,count_scaled,rank
albums_id,albums,,,
1kS7q9KBO8X6xfGffRcYaH,A State Of Trance Year Mix 2018 (Mixed by Armin van Buuren),11800,1.00,1.00
7fAZC9BKYwEqZsCc488QHl,A State Of Trance Year Mix 2016 (Mixed by Armin van Buuren),1649,0.14,2.00
4HvKkLUMiYoQfqQ8wDGzJL,The United States of Russia,1620,0.14,3.00
66Rs4ct2c0Y6Ao9ICN7RhY,The Fresh Prince of Donair,810,0.07,4.00
4gTOSlnyuvjaN5D7S7NfdD,Ok,675,0.06,5.00
...,...,...,...,...
1yx0lKqG8Z1J3IOxx4gbzs,Nina,1,0.00,4953.50
1W9toxqtPfieKk6cft0f7R,Streets Of Gold,1,0.00,4953.50
1hwjIPHrnNBFuxWBWRHmge,'Di Na Muli,1,0.00,4953.50


In [10]:
df_artist.head
artist_rank = df_artist.groupby(['artists_id','artists']).agg(artist_count=('artists_id','count'))

min_max = MinMaxScaler()
artist_rank['count_scaled'] = min_max.fit_transform(artist_rank[["artist_count"]])

artist_rank["rank"] = artist_rank['artist_count'].rank(method='average',ascending=False)

artist_rank = artist_rank.sort_values('artist_count',ascending=False)

artist_rank

,,artist_count,count_scaled,rank
artists_id,artists,,,
0SfsnGyD8FpIN4U4WCkBZ5,Armin van Buuren,13691,1.00,1.00
0LyfQWJT6nXafLPZqxe9Of,Various Artists,9129,0.67,2.00
3Nrfpe0tUJi4K4DXYWgMUX,BTS,1884,0.14,3.00
39Bn2Hd2sBOQBFRR3jIzEV,Ben Rosenfeld,1620,0.12,4.00
4VMYDCV2IEDYJArk749S6m,Daddy Yankee,1331,0.10,5.00
...,...,...,...,...
3wbCeEPAW6po7J46netxMT,Up Dharma Down,1,0.00,3073.50
2U5mF0PZqGu6glnz55yY0y,John Roa,1,0.00,3073.50
0kkxsdcaWmWU2yWAqclDh4,Rixton,1,0.00,3073.50


In [11]:
# tracks_cartesian = []
# for element in itertools.product(s_tracks_id,s_tracks_id):
#     tracks_cartesian.append(element)

In [12]:
# https://stackoverflow.com/questions/11144513/cartesian-product-of-x-and-y-array-points-into-single-array-of-2d-points
# matrix track x track

def cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(numpy.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)


# tracks_cartesian = cartesian_product(*(s_tracks_id * 2))

In [13]:
# tracks_cartesian

In [14]:
playlist = sp.search(q='year:2018', type='playlist', limit=50, offset=62)
for user in playlist['playlists']['items']:
    pid = user['id']
    uid = user['owner']['id']
    
p_user_id, p_playlist_id, p_artists, p_albums, p_tracks, p_popularity, p_artists_id, p_albums_id, p_tracks_id = get_playlist_detail(uid,pid)

In [15]:
print(playlist)

{'playlists': {'href': 'https://api.spotify.com/v1/search?query=year%3A2018&type=playlist&offset=62&limit=50', 'items': [{'collaborative': False, 'description': 'Masters of Hardcore | 2018 Yearmix by MoH themselves. Inspired by mix @ https:&#x2F;&#x2F;www.youtube.com&#x2F;watch?v=QVs-TFuUGrg', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/4JOE9J4JvJStZK5KNzfjR2'}, 'href': 'https://api.spotify.com/v1/playlists/4JOE9J4JvJStZK5KNzfjR2', 'id': '4JOE9J4JvJStZK5KNzfjR2', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab67706c0000bebbd875cb9adcff4d9575e9ec0b', 'width': None}], 'name': 'Masters of Hardcore | 2018 Yearmix', 'owner': {'display_name': 'Johan Becker', 'external_urls': {'spotify': 'https://open.spotify.com/user/twistahkid'}, 'href': 'https://api.spotify.com/v1/users/twistahkid', 'id': 'twistahkid', 'type': 'user', 'uri': 'spotify:user:twistahkid'}, 'primary_color': None, 'public': None, 'snapshot_id': 'NTYsNzAxYjg4ZDcxYmQzZjhlMzA4Y2FjM2IzNTE3MTcxNzRiM

In [16]:
for item in playlist:
    print(item)

playlists


In [17]:
for item in playlist['playlists']['items']:
#     print(item)
    for a in item:
        print(a)

collaborative
description
external_urls
href
id
images
name
owner
primary_color
public
snapshot_id
tracks
type
uri
collaborative
description
external_urls
href
id
images
name
owner
primary_color
public
snapshot_id
tracks
type
uri
collaborative
description
external_urls
href
id
images
name
owner
primary_color
public
snapshot_id
tracks
type
uri
collaborative
description
external_urls
href
id
images
name
owner
primary_color
public
snapshot_id
tracks
type
uri
collaborative
description
external_urls
href
id
images
name
owner
primary_color
public
snapshot_id
tracks
type
uri
collaborative
description
external_urls
href
id
images
name
owner
primary_color
public
snapshot_id
tracks
type
uri
collaborative
description
external_urls
href
id
images
name
owner
primary_color
public
snapshot_id
tracks
type
uri
collaborative
description
external_urls
href
id
images
name
owner
primary_color
public
snapshot_id
tracks
type
uri
collaborative
description
external_urls
href
id
images
name
owner
primary_color


In [18]:
# print(playlist['playlists']['items'])
a = playlist['playlists']['items']

type(a)
# for i in a.keys():
#     print(i)

list